In [9]:
# Transfer Learning solution

Certainly! This program demonstrates a common workflow in deep learning for image classification using transfer learning with a pre-trained model on the CIFAR-10 dataset.

High-level concept:

The core idea is to leverage a powerful, pre-trained neural network (ResNet-18) that has already learned to recognize features from a very large dataset (ImageNet). Instead of training a new model from scratch, which requires a lot of data and computational resources, we adapt this pre-trained model for a new, but related, task (CIFAR-10 image classification).

We 'freeze' most of the pre-trained model's layers, meaning their learned weights won't change during training. We then replace only the final classification layer with a new one tailored to the 10 classes of CIFAR-10. This new layer, along with any unfrozen parts of the original model (if any), is then trained on the CIFAR-10 dataset. This approach is called transfer learning, and it's very effective for tasks where you have limited data but a relevant pre-trained model exists.

What the program is doing step-by-step:

Setup and Data Preparation:

Installs torch, torchvision, and torchaudio libraries.
Imports necessary modules from PyTorch for building and training neural networks.
Defines image transforms to resize images and convert them to PyTorch tensors.
Downloads and loads the CIFAR-10 dataset, splitting it into training and testing sets, and applies the defined transformations.
Creates DataLoader objects to efficiently load data in batches during training and evaluation.
Model Loading and Modification (Transfer Learning):

Loads a pre-trained ResNet-18 model (a popular convolutional neural network architecture).
Freezes all the layers of the loaded ResNet-18, preventing their weights from being updated during training.
Replaces the model's final fully connected (fc) layer (which originally classified 1000 ImageNet classes) with a new linear layer that outputs 10 classes, matching CIFAR-10.
Moves the model to the appropriate device (cuda for GPU if available, otherwise cpu).
Training Setup:

Defines the CrossEntropyLoss function, which is commonly used for multi-class classification problems.
Sets up the Adam optimizer, which will be used to update the weights of the newly added classification layer (since other layers are frozen).
Training Loop:

Iterates for a specified number of epochs (in this case, 2).
In each epoch, it goes through the train_loader batch by batch:
Moves images and labels to the device.
Performs a forward pass (makes predictions).
Calculates the loss between predictions and actual labels.
Performs a backward pass (calculates gradients).
Updates the model's parameters (only the new fc layer's weights).
Prints the average training loss for each epoch.
Evaluation:

Sets the model to evaluation mode (model.eval()).
Disables gradient calculations (torch.no_grad()) to save memory and speed up computation.
Iterates through the test_loader batch by batch:
Makes predictions on the test data.
Compares predictions to actual labels to count correct classifications.
Calculates and prints the final accuracy of the model on the test dataset.

In [1]:
!pip install torch torchvision torchaudio --quiet

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [03:08<00:00, 907kB/s] 


In [5]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 226MB/s]


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [7]:
epochs = 2

for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/2] Loss: 0.8367
Epoch [2/2] Loss: 0.6221


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = outputs.max(1)

        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {100 * correct / total:.2f}%")


Accuracy: 79.54%
